# Демография РФ (1990 - 2017гг)

В данной работе реализована визуализация демографических изменнений по субъектам РФ за период с 1990 по 2017гг.
Для каждого субъекта (региона, области и т.п.) можно посмотреть количество родившихся, умерших и прирост насления
на 1000 жителей в динамике по годам с 1990 по 2017.

Источник данных: https://www.kaggle.com/dwdkills/russian-demography

Другие полезные ссылки, которые я хочу сохранить здесь для себя:

библиотека виджетов https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html#Dropdown

построение разных графиков https://pyprog.pro/mpl/mpl_bar.html


#### 1. Загрузка данных.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

df = pd.read_csv("russian_demography.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2380 entries, 0 to 2379
Data columns (total 7 columns):
year            2380 non-null int64
region          2380 non-null object
npg             2317 non-null float64
birth_rate      2317 non-null float64
death_rate      2315 non-null float64
gdw             2330 non-null float64
urbanization    2330 non-null float64
dtypes: float64(5), int64(1), object(1)
memory usage: 130.2+ KB


#### 2. Поиск пропущенных значений.

In [2]:
def None_mask_clmn(df, columns):
    if len(columns) == 0:
        return "Error: no column found."
    mask = df[columns[0]].isnull()
    for col in columns[1:]:
        mask = mask | df[col].isnull()
    return mask

In [3]:
#регионы по которым неполная информация
problem_df = df.loc[None_mask_clmn(df, df.columns)]
problem_df['region'].unique()

array(['Republic of Ingushetia', 'Republic of Crimea', 'Sevastopol',
       'Chechen Republic'], dtype=object)

In [4]:
#df.loc[df['region'] == 'Republic of Crimea']
#df.loc[df['region'] == 'Republic of Ingushetia']
#df.loc[df['region'] == 'Sevastopol']
#df.loc[df['region'] == 'Chechen Republic']

#### (Попытка разобраться с визуализацией карты с регионами)

Здесь я пробовал понять как можно делать heatmap'ы на реальных географических картах, но у меня не получилось даже понять как загрузить карту регионов РФ. В частности, не получилось установить geopandas.)

In [5]:
import geopandas as gpd
%matplotlib inline
reg_gdf = gpd.read_file('Regions.shp')
reg_gdf.head()

ModuleNotFoundError: No module named 'geopandas'

#### 3. Перевод названий субъектов в отдельном .xlsx файле.

In [6]:
reg_df = pd.DataFrame(np.sort(df['region'].unique()))

## save to xlsx file

filepath = 'reg.xlsx'

reg_df.to_excel(filepath, index=False)
reg_df_upd = pd.read_excel('reg_upd.xlsx', index=False)
reg_df_upd.sort_values(by = ['region_rus'], inplace = True)
#reg_df_upd['region_eng']
#reg_df_upd["region_rus_eng"] = 
region_dict = np.array(reg_df_upd.apply(lambda x: (x['region_rus'], x['region_eng']), axis=1))
#reg_df_upd.head()

#### 4. Функция для построения графика по одному региону.

In [7]:
# Здесь была функция побольше, но потом мне она стала не нужна и от неё остался такой вот обрубок)
def select_region(reg):
    return df.loc[df['region'] == reg]

In [24]:
# функция для посторения гистограммы по выбранному региону 
def plot_gist(reg):
    msk_df = select_region('Moscow')
    reg_df = select_region(reg)
    
    width = 0.75 # ширина столбцов гистограммы
    year = np.array(msk_df['year'])
    pop_growth = np.array(reg_df['npg'])
    birth_rate = np.array(reg_df['birth_rate'])
    death_rate = np.array(reg_df['death_rate'])*(-1)

    fig, ax = plt.subplots()
    ax.set_xlabel('годы')
    ax.set_ylabel('человек на 1000 жителей')
    ax.plot(year, pop_growth, label="прирост", color = 'blue')
    ax.bar(year, death_rate, width, label="смертность", color='#d95f02')
    ax.bar(year, birth_rate, width, label="рождаемость", color = '#1b9e77')
    ax.legend() # вывести легенду
    fig.suptitle(reg + ' demography') # название Figure
    fig.set_figwidth(12)    #  ширина Figure
    fig.set_figheight(8)    #  высота Figure
    plt.show()

#### 5. Виджет для выпадающего списка и результирующая картинка.

In [25]:
import ipywidgets as wdg

def printout(reg):
    print(reg)
    
wdg.interact(plot_gist, reg = wdg.Dropdown(
    options=region_dict,
    value='Moscow',
    description='Субъект РФ:',
    disabled=False,
)
)

interactive(children=(Dropdown(description='Субъект РФ:', index=84, options=(('Алтайский край', 'Altai Krai'),…

<function __main__.plot_gist(reg)>